In [3]:
import requests
import os
from os import mkdir
from os.path import join, exists
import json
from time import sleep
from tqdm import tqdm 

from fake_useragent import UserAgent

# urls to base webpages on sg.ch

base = [
    "https://www.sg.ch/ueber-den-kanton-st-gallen.html",
    "https://www.sg.ch/politik-verwaltung.html",
    "https://www.sg.ch/bauen.html",
    "https://www.sg.ch/bildung-sport.html",
    "https://www.sg.ch/gesundheit-soziales.html",
    "https://www.sg.ch/kultur.html",
    "https://www.sg.ch/recht.html",
    "https://www.sg.ch/sicherheit.html",
    "https://www.sg.ch/steuern-finanzen.html",
    "https://www.sg.ch/umwelt-natur.html",
    "https://www.sg.ch/verkehr.html",
    "https://www.sg.ch/wirtschaft-arbeit.html",    
]

data_dir = "data"

ua = UserAgent()

In [4]:
from os import makedirs
from os.path import join
from bs4 import BeautifulSoup

# collecting a mapping from dir withiin `data_dir` to source link to the HTML page from sg.ch

all_source2base_ = {}

def recursive_scrap(links: list, all_source2base: dict):
    source2base = {
        link.replace("https://www.sg.ch/", "").replace(".html", ""): link 
        for link in links
    }
    all_source2base = {**all_source2base, **source2base}
    for source, link in source2base.items():
        if not exists(join(data_dir, source)):
            makedirs(join(data_dir, source))
        data_path = join(data_dir, source, "data.html")
        if not exists(data_path):
            with open(data_path, "w") as f:
                try:
                    content = requests.get(
                        link, 
                        headers={'User-Agent': ua.chrome} # to avoid the ban 
                    ).text
                except:
                    sleep(10)
                    continue
                f.write(content)
                sleep(0.2)
        subpages = []
        with open(data_path, "r") as f:
            content = f.read()
            soup = BeautifulSoup(content, 'html.parser')
            for e in soup.find_all("a"):
                href = e.attrs.get("href", "")
                if f"/{source}/" in href and ".html" in href:
                    if "/content/sgch/" in href:
                        href = href.replace("content/sgch/", "")
                    subpages.append(
                        f"https://www.sg.ch{href}" 
                        if "https://www.sg.ch" not in href 
                        else href
                    )
            subpages = list(set(subpages))
            all_source2base = recursive_scrap(subpages, all_source2base)
    return all_source2base

# the only reason to do such loop is to monitor progress via tqdm
for url in tqdm(base):
    all_source2base_ = recursive_scrap([url], all_source2base=all_source2base_)

In [16]:
if not exists("sourve2link.jsonl"):
    with open("sourve2link.jsonl", "w") as f:
        for path_to_data, link in tqdm(all_source2base_.items()):
            path = join(data_dir, path_to_data, "data.html")
            voice_url = None
            with open(path, "r") as g:
                soup = BeautifulSoup(g.read(), 'html.parser')
                for e in soup.find_all("a"):
                    href = e.attrs.get("href", "")
                    if "//app-eu.readspeaker.com/cgi-bin" in href: # the url to the voice recording
                        voice_url = f"https:{href.replace('amp;','')}"
            f.write(
                json.dumps({
                    "source": link, 
                    "path": join(data_dir, path_to_data), 
                    "voice_url": voice_url
                }) + "\n")

data = []
with open("sourve2link.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3879/3879 [06:27<00:00, 10.01it/s]


In [12]:
import pandas as pd
pd.DataFrame(data).to_csv("data.csv")